In [ ]:
!pip install transformers
!pip install scikit-learn  -U

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from IPython.display import clear_output
import transformers as ppb
import warnings
import pandas
import torch
import numpy

warnings.filterwarnings('ignore')
clear_output()
chat = pandas.read_csv('../input/chatbot-dataset-topical-chat/topical_chat.csv')
chat

In [ ]:
# Vamos usar só as primeiras 2000 frases
batch = chat[:2000]
batch['sentiment'].value_counts()

In [ ]:
# Vamos puxar o DistilBert
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Carrega o tokenizer pré-treinado
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)
clear_output()

In [ ]:
tokenized = batch['message'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = numpy.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = numpy.where(padded != 0, 1, 0)
numpy.array(padded).shape

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()
labels = batch['sentiment']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(train_features, train_labels)
lr_clf.score(test_features, test_labels)

In [ ]:
from sklearn.metrics import confusion_matrix
pred = lr_clf.predict(test_features)
confusion_matrix(test_labels, pred)